In [1]:
import requests

# Post

In [19]:
server_url = 'http://183.99.13.124:1080'
upload_url=server_url + '/upload'

# 키 전달
def set_key():
    # 미리 정해져있음 
    flists = [("enc_key", 'EncKey.txt'), ('mul_key', 'MulKey.txt')]
    for dtype, fn in flists:
        r = requests.post(upload_url, 
                          files={'file':open(fn, 'rb')}, 
                          headers={'dtype':dtype})
        if not r.ok:
            print("ERROR")
            return -1
    return True

# 암호문 전달
def send_ctxt(action, ori):
    """e.g., ctx_05_e.dat"""
    files = {'file': open(f'./ctx_{action:02}_{ori}.dat','rb')}
    header = {'dtype':"ctxt", "action":f'{action}'}
    r = requests.post(upload_url, files=files, headers=header)
    return r

In [10]:
r = requests.post(upload_url, 
  files={'file':open('EncKey.txt', 'rb')}, 
  headers={'dtype':'enc_key'})

In [6]:
r = set_key()

In [18]:
r.request.body

b'--f87a4683e819c93b23c185f61c513a1b\r\nContent-Disposition: form-data; name="file"; filename="EncKey.txt"\r\n\r\nsd;kjfdslfjsldhdsvk\n\r\n--f87a4683e819c93b23c185f61c513a1b--\r\n'

In [12]:
print(r.request.url)
print(r.request.headers)

http://183.99.13.124:1080/upload
{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'dtype': 'enc_key', 'Content-Length': '166', 'Content-Type': 'multipart/form-data; boundary=f87a4683e819c93b23c185f61c513a1b'}


In [17]:
r.request.headers.keys

<bound method Mapping.keys of {'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'dtype': 'enc_key', 'Content-Length': '166', 'Content-Type': 'multipart/form-data; boundary=f87a4683e819c93b23c185f61c513a1b'}>

In [13]:
for kk, dd in r.request.headers:
    print(kk, dd)

ValueError: too many values to unpack (expected 2)

In [20]:
r = send_ctxt('13', 'a')

In [22]:
r.request.headers

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive', 'dtype': 'ctxt', 'action': '13', 'Content-Length': '166', 'Content-Type': 'multipart/form-data; boundary=a797912a6e8d63365def0ed0324cba1c'}

# get

서버에 계산된 암호문 파일 받은 후 동형암호 라이브러리로 읽어서 복호화, 결과 확인.

**서버 계산이 완료될 때 까지 5초에 한번씩 get 시도**


In [14]:
from time import sleep

def save_binary(r, fn_save):
    if r.status_code == 200:
        with open(fn_save, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        raise FileNotFoundError

result_url=server_url + '/result'

def get_result(result_url):

    while True:
        r = requests.get(result_url, stream=True)
        if r.status_code == 200:
            save_binary(r,'./client_results/pred0.dat')
            print("Result recieved.")
            break
        else:
            sleep(5)


In [20]:
ls './client_results'

pred0.dat


### 결과 확인 예시

In [ ]:
# HEAAN 라이브러리로 ctxt 파일 로드
logp = 30
logq = 300
nslots = 2**14
ctx_pred = he.Ciphertext(logp, logq, nslots)
he.SerializationUtils.readCiphertext(ctx_pred, fn_ctx)
# decryptor로 decrypt
dec=decrypt(self.scheme, self.secretKey, ctx_pred, self.parms)
# 결과 출력 
print(np.argmax(dec))